In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot

import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LSTM
from keras import backend as K

import gc

In [3]:
def cnn_data_sequence_pre_train(data, target, seque_len, previo_aniadido=False):
    print(data.shape)
    print(target.shape)
    new_data = []
    new_target = []
    if (previo_aniadido == False):
        for index in range(data.shape[0] - seque_len + 1):
            new_data.append(data[index: index + seque_len])
            new_target.append(target[index + seque_len - 1])
            print(index, ':', index + seque_len - 1, '-->', index + seque_len - 1)
    else:
        for index in range(data.shape[0] - seque_len + 1):
            new_data.append(data[index: index + seque_len])
            new_target.append(target.iloc[index])
            print(index, ':', index + seque_len - 1, '-->', index)

    new_data = np.array(new_data)
    new_target = np.array(new_target)

    new_data = new_data.reshape(new_data.shape[0], new_data.shape[1], new_data.shape[2], 1)

    return new_data, new_target


def mostrarEvolucion(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    pyplot.plot(loss)
    pyplot.plot(val_loss)
    pyplot.legend(['Training loss', 'Validation loss'])
    pyplot.show()

    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']
    pyplot.plot(acc)
    pyplot.plot(val_acc)
    pyplot.legend(['Training accuracy', 'Validation accuracy'])
    pyplot.show()


### Cargamos el conjunto de datos
# Cargamos los datos originales
df = pd.read_csv('../spyconalfas.csv')
# Separamos en X - (conjunto de entrenamiento) y Y - (etiquetas de los movimientos)
Y = df['next movement']
X = df
X.drop(['next movement', 'alpha53'], axis=1, inplace=True)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

fecha_inicio_train = X_train['Date'].iloc[0]
fecha_fin_train = X_train['Date'].iloc[-1]
fecha_inicio_test = X_test['Date'].iloc[0]
fecha_fin_test = X_test['Date'].iloc[-1]

print('Intervalo de entrenamiento:', fecha_inicio_train, '-', fecha_fin_train)
print('Intervalo de prueba:', fecha_inicio_test, '-', fecha_fin_test)

X_train_open = X_train['Open']
X_test_open = X_test['Open']
X_train_close = X_train['Close']
X_test_close = X_test['Close']

X_train_ret = X_train['next returns']
X_test_ret = X_test['next returns']
X_train.drop(['next returns', 'Date', 'Open'], axis=1, inplace=True)
X_test.drop(['next returns', 'Date', 'Open'], axis=1, inplace=True)

cols_to_norm = ['Close', 'alpha54', 'alpha84']
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])

n_caracteristicas = X_train.shape[1]

X_train_last_59 = X_train.iloc[-59:].copy()
X_test = pd.concat([X_train_last_59, X_test])
print(X_test.shape)
print(Y_test.shape)
X_train, Y_train = cnn_data_sequence_pre_train(X_train, Y_train, seque_len=60)
X_test, Y_test = cnn_data_sequence_pre_train(X_test, Y_test, seque_len=60, previo_aniadido=True)

Intervalo de entrenamiento: 2013-10-15 - 2021-01-06
Intervalo de prueba: 2021-01-07 - 2022-10-13
(496, 84)
(437,)
(1747, 84)
(1747,)
0 : 59 --> 59
1 : 60 --> 60
2 : 61 --> 61
3 : 62 --> 62
4 : 63 --> 63
5 : 64 --> 64
6 : 65 --> 65
7 : 66 --> 66
8 : 67 --> 67
9 : 68 --> 68
10 : 69 --> 69
11 : 70 --> 70
12 : 71 --> 71
13 : 72 --> 72
14 : 73 --> 73
15 : 74 --> 74
16 : 75 --> 75
17 : 76 --> 76
18 : 77 --> 77
19 : 78 --> 78
20 : 79 --> 79
21 : 80 --> 80
22 : 81 --> 81
23 : 82 --> 82
24 : 83 --> 83
25 : 84 --> 84
26 : 85 --> 85
27 : 86 --> 86
28 : 87 --> 87
29 : 88 --> 88
30 : 89 --> 89
31 : 90 --> 90
32 : 91 --> 91
33 : 92 --> 92
34 : 93 --> 93
35 : 94 --> 94
36 : 95 --> 95
37 : 96 --> 96
38 : 97 --> 97
39 : 98 --> 98
40 : 99 --> 99
41 : 100 --> 100
42 : 101 --> 101
43 : 102 --> 102
44 : 103 --> 103
45 : 104 --> 104
46 : 105 --> 105
47 : 106 --> 106
48 : 107 --> 107
49 : 108 --> 108
50 : 109 --> 109
51 : 110 --> 110
52 : 111 --> 111
53 : 112 --> 112
54 : 113 --> 113
55 : 114 --> 114
56 : 11

In [5]:
model = Sequential()

model.add(LSTM(units=60,return_sequences=True))
model.add(LSTM(84, return_sequences=True))
model.add(LSTM(84))
model.add(Dense(1,activation = 'sigmoid'))

In [6]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [8]:
model.fit(X_train,Y_train)

ValueError: in user code:

    File "c:\users\judith\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\users\judith\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\judith\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\judith\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\users\judith\appdata\local\programs\python\python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\users\judit